In [1]:
import re
import os
import collections
import time
import codecs

import requests
import pandas as pd
import ftplib
import io
import re
import json
import datetime

try:
    from requests_html import HTMLSession
except Exception:
    print("""Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.""")
    
import csv
import pandas as pd
from progressbar import progressbar

from io import StringIO
from html.parser import HTMLParser

C:\Users\srema\AppData\Local\Continuum\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# HTML cleaner
# Source: https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python/925630#925630

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    ''' Strip HTML tags from text'''
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
# Scraping data and saving as JSON (new site)

def get_html_text(url):
    ''' Send HTML request and return HTML text in UTF-8 format'''
    
    session = HTMLSession()
    resp = session.get(url)
    text = resp.html.raw_html.decode("utf-8")  
    session.close()   
    
    return text

def remove_emojis(text):
    ''' Remove emojis and special characters'''
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    
    no_emoji = emoji_pattern.sub(r'', text) 
    return (no_emoji.encode('ascii', 'ignore')).decode("utf-8")

def parse_html(url):
    ''' Fetch HTML and clean it  to get raw JSON string'''
    
    text = get_html_text(url)
    text_split = text.split('\n')
    test_str = strip_tags(text_split[-21])
    raw_data = test_str[test_str.find('{"props"'):]
    cleaned_raw_data = remove_emojis(raw_data)
    
    return cleaned_raw_data

def save_json(path, filename, text):
    ''' Save scraped text as JSON in UTF-8 format'''
    
    text_file = codecs.open(f"{path}/{filename}.json", "w", "utf-8")
    n = text_file.write(text)
    text_file.close()
    
def get_filename(url):
    ''' Get filename from URL'''
    year_month = url.split('/')
    return year_month[4][:6]
    
def scrape_json(json_path, url_path):
    ''' Scrape URL and save it as JSON'''
    
    url_list = open_csv(url_path)
    
    for url in progressbar(url_list):
        time.sleep(10)
        raw_data = parse_html(url)
        filename = get_filename(url)
        save_json(json_path, filename, raw_data)

In [4]:
# Parsing data from JSON files (new site)

def flatten(d, parent_key='', sep='_'):
    ''' Flatten dictionary'''
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def clean_data(df):
    ''' Adds extra colums to dataframe and renames some columns'''
    
    df_clean = df.copy()
    df.columns = df.columns.str.strip('quote')
    df_clean['Ticker'] = df_clean['symbol'] + '-USD'
    df_clean['Date'] = pd.to_datetime(df_clean['quoteusdlastupdated']).dt.date
    df_clean = df_clean.rename(columns={"id": "ID", "nam": "Name", "quoteusdmarketcap": "Market_cap"})
    
    
    return df_clean 

def remove_list_from_dict(dictionary):
    ''' Remove columns from dictionary that are not in the list'''
    
    key_list = ['id', 'name', 'symbol', 'slug', 'cmcrank', 'marketpaircount', 'circulatingsupply',
                'totalsupply', 'lastupdated', 'dateadded', 'tags', 'rank',  
                'quoteusdname', 'quoteusdprice', 'quoteusdvolume24h', 
                'quoteusdmarketcap', 'quoteusdpercentchange1h', 'quoteusdpercentchange24h', 
                'quoteusdpercentchange7d', 'quoteusdlastupdated']
    
    copy_dict = dictionary.copy()
    
    for key in dictionary.keys():
        if key not in key_list:
            del copy_dict[key]
    
    return copy_dict

def format_keys(dictionary):
    ''' Convert keys to lowercase and remove underscore'''
    
    old_keys = dictionary.keys()
    new_dict = {}
    
    for key, value in dictionary.items():
        new_key = key.lower().replace("_", "")
        new_dict[new_key] = value
    
    return new_dict
    
def parse_json(file_path):
    ''' Parses JSON file and converts it into a dataframe. 
    
        The JSON structure is loaded as a dictionary and then flatted 
        after which the data parsed into a dataframe
        
        params: 
            file_path: string - Directory if JSON files
        
        returns:
            df_clean: pandas - Pandas dataframe with cleaned data
    '''
    
    row_values = []
     
    col_names = []
    
    with open(file_path) as json_file:
        json_data = json.load(json_file)
    
    json_dict = json_data['props']['initialState']['cryptocurrency']['listingLatest']['data']

    for i in range(len(json_dict)):
        
        flat_dict = format_keys(flatten(json_dict[i], parent_key='', sep='_'))
        clean_dict = remove_list_from_dict(flat_dict)
        item_list = list(clean_dict.values())

        row_values.append(item_list)
        
        if len(col_names) == 0:
            col_names = list(clean_dict.keys())
    

    df_raw = pd.DataFrame(row_values, columns=col_names)
                
    df_clean = clean_data(df_raw)
    
    return df_clean

In [5]:
# Scraping and parsing the old site

def get_date(url):
    ''' Return date from URL'''
    
    split_url = url.split('/')
    raw_date = split_url[4][:8]
    return f"{raw_date[:4]}-{raw_date[4:6]}-{raw_date[6:]}"
    
def list_to_dataframe(raw_data, url, columns):
    ''' Convert list of lists with scraped data to a dataframe and add some extra columns'''
    
    
    
    df = pd.DataFrame.from_records(raw_data, columns=columns)
    df['Ticker'] = df['symbol'] + '-USD'
    df['Date'] = get_date(url)
    df['Date'] = pd.to_datetime(df['Date'])
    
    return df


def find_url(path, filename):
    ''' Find the URL of the file that could not be parsed by looking for a substring in the URL list'''
    
    url_list = open_csv(path)
    date = filename.replace(".json", "")
    index = [i for i, s in enumerate(url_list) if date in s][0]
    
    return url_list[index]

def text_cleaner(text):
    ''' Remove duplicates from text'''
    
    clean_text = text

    for i in range(len(text)):
        is_int = False
        try:
            int(text[i])
            is_int = True
        except:
            pass
        
        if is_int:
            unique_item = text[i+1]
            if unique_item == text[i+6]:
                del text[i+6]
    
    return clean_text
    
def parse_old_site_2016(url_path, filename):
    ''' Scrape and parse the old site and return a pandas dataframe
    
        This funtion is triggered if parsing the JSON file from the "scrape_json" could not be parsed.
        The function first scrapes the intended page, then removes all the HTML so that 
        only a list with the data remains. This list is then index and written to a dataframe.
        
        params:
            url_path: string - Directory of the URL list
            filename: string - Name of the JSON file that could not be parsed
        
        returns:
            df: pandas dataframe - dataframe with scraped data            
    '''
    
    url = find_url(url_path, filename)
    text = get_html_text(url)
    
    stripped_text = strip_tags(text).split('\n')
    begin_index = stripped_text.index('                    % Change (24h)Price Graph (7d)')
    end_index = stripped_text.index('                        Next 100  →')
    indexed_text = stripped_text[begin_index:end_index] 

    spaceless_text = [item.replace(" ", "") for item in indexed_text]
    nostar_text = [item.replace("*", "") for item in spaceless_text]
    not_empty_text = [item for item in nostar_text if len(item)>0]

    raw_data = []
    for i in range(1, len(not_empty_text), 8):
        raw_data.append(not_empty_text[i:i+8])
        
    columns = ['rank', "Name", "Market_cap", "USD_pric", 
               "Available_supply", "symbol", "USD_volume_24h", "USD_percent_change_24h"]
    
    df = list_to_dataframe(raw_data, url, columns)

    return df

def parse_old_site_2017(url_path, filename):
    ''' Scrape and parse the old site and return a pandas dataframe
    
        This funtion is triggered if parsing the JSON file from the "scrape_json" could not be parsed.
        The function first scrapes the intended page, then removes all the HTML so that 
        only a list with the data remains. This list is then index and written to a dataframe.
        
        params:
            url_path: string - Directory of the URL list
            filename: string - Name of the JSON file that could not be parsed
        
        returns:
            df: pandas dataframe - dataframe with scraped data            
    '''
    
    url = find_url(url_path, filename)
    text = get_html_text(url)
    stripped_text = strip_tags(text).split('\n')
    spaceless_text = [item.replace(" ", "") for item in stripped_text]
    nostar_text = [item.replace("*", "") for item in spaceless_text]
    not_empty_text = [item for item in nostar_text if len(item)>0]
    try:
        begin_index = not_empty_text.index('%Change(24h)PriceGraph(7d)')
    except:
        begin_index = not_empty_text.index('PriceGraph(7d)')

    end_index = not_empty_text.index('NotMineable')

    indexed_text = not_empty_text[begin_index:end_index-2] 

    clean_text = text_cleaner(indexed_text)

    raw_data = []
    for i in range(1, len(clean_text), 9):
        raw_data.append(clean_text[i:i+9])
        
    columns = ['rank', "Ticker","Name", "Market_cap", "USD_pric", "USD_volume_24h",
               "Available_supply", "symbol", "USD_percent_change_24h"]
    
    df = list_to_dataframe(raw_data, url, columns)

    return df

In [6]:
def open_csv(path):
    ''' Open a CSV and return a list with the contents'''

    with open(path, newline='') as f:
        reader = csv.reader(f)
        url_list = list(reader)[1:]
        
    return [url[0] for url in url_list]


def save_df(json_path, url_path):
    ''' Open JSON json files and call the "parse_json" function
    
        Some JSON files are empty because the old CoinMarketCap website cannot be 
        scraped in the same way as the new one. For the old site I made a seperate 
        function that both scrapes and parses the data.
        
        params:
        
            json_path: string - Directory of the JSON files
            url_path: string - Directory of the URL list
        
    '''
    is_2017 = False
    
    for file in progressbar(os.listdir(json_path)):
        file_path = f"{json_path}/{file}"
        filename = file.replace(".json", ".csv")
        print(filename)
        
        try:
            df = parse_json(file_path)
            df_clean = clean_data(df)
            df_clean.to_csv(f"data/market_cap/{filename}", index=False)

        except json.decoder.JSONDecodeError:
            
            if filename == "201710.csv":
                is_2017 = True
            if is_2017:
                df_old_site_2017 = parse_old_site_2017(url_path, file)
                df_old_site_2017.to_csv(f"data/market_cap/{filename}", index=False)
            else: 
                df_old_site_2016 = parse_old_site_2016(url_path, file)
                df_old_site_2016.to_csv(f"data/market_cap/{filename}", index=False)
        
        except AttributeError:
            print('NOOO:', filename)
   

In [7]:
url_path = 'data/market_cap/urls.csv'

json_path = "data/json"
# scrape_json(json_path, url_path)
save_df(json_path, url_path)

N/A% (0 of 61) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

201612.csv


  1% (1 of 61) |                         | Elapsed Time: 0:00:03 ETA:   0:03:49

201701.csv


  3% (2 of 61) |                         | Elapsed Time: 0:00:06 ETA:   0:03:06

...

201912.csv
202001.csv
202002.csv
202003.csv
202004.csv
202005.csv
202006.csv
202007.csv
202008.csv

 85% (52 of 61) |####################    | Elapsed Time: 0:02:00 ETA:   0:00:02


202009.csv
202010.csv
202011.csv
202012.csv
202101.csv
202102.csv
202103.csv
202104.csv
202105.csv


100% (61 of 61) |########################| Elapsed Time: 0:02:00 Time:  0:02:00


202106.csv
202107.csv
202108.csv
202109.csv
202110.csv
NOOO: 202110.csv
202111.csv
NOOO: 202111.csv
202112.csv
NOOO: 202112.csv
